In [1]:
import numpy as np
import awkward as ak
import warnings
import matplotlib.pyplot as plt
import hist
import math
import os
import json
import gc
import dask_awkward as dak
import mplhep as hep
from scipy.stats import kstest

In [2]:
warnings.filterwarnings('ignore', 'invalid value')
warnings.filterwarnings('ignore', 'divide by')
warnings.filterwarnings('ignore', 'overflow encountered')
warnings.filterwarnings('ignore', 'Conversion of an array')

In [3]:
path = '/scratch365/cmoore24/training/hgg/batch2024/ml_results_checking'
with open(f'{path}/subregion_event_totals.json', 'r') as f:
    totals = json.load(f)

In [4]:
with open(f'{path}/my_xsecs.json', 'r') as f:
    xsecs = json.load(f)

In [5]:
xsecs2 = {}
for i in xsecs:
    if type(xsecs[i]) == dict:
        for j in xsecs[i]:
            xsecs2[j] = xsecs[i][j]
    else:
        xsecs2[i] = xsecs[i]
xsecs = xsecs2

In [6]:
totals2 = {}
for i in totals:
    if type(totals[i]) == dict:
        for j in totals[i]:
            totals2[f'{i}_{j}'] = totals[i][j]
    else:
        totals2[i] = totals[i]
totals = totals2

In [7]:
def read_files(input_path):
    if ('.parquet' in os.listdir(input_path)[0]):
        output = ak.from_parquet(f'{input_path}/keep*', columns=columns)
    else:
        output = {}
        for i in os.listdir(input_path):
            if ('flat400' not in i):
                output[i] = ak.from_parquet(f'{input_path}/{i}/keep*', columns=columns)
            else:
                continue
    return output

In [8]:
def add_ratio(ratio, dataframe):
    dash = ratio.find('/')
    asterisk = ratio.find('*')
    numerator = ratio[:dash]
    denominator = ratio[dash+1:asterisk]
    exponent = float(ratio[asterisk+2:])
    num_ecf = dataframe[numerator]
    den_ecf = dataframe[denominator]
    ecf_ratio = (num_ecf / (den_ecf**exponent))   
    return ecf_ratio

In [9]:
def get_num_den(ratio):
    dash = ratio.find('/')
    asterisk = ratio.find('*')
    numerator = ratio[:dash]
    denominator = ratio[dash+1:asterisk]  
    return (numerator, denominator)

In [10]:
def firsts(mc):
    for i in mc:
        if type(mc[i]) == dict:
            for j in mc[i]:
                for k in mc[i][j].fields:
                    if 'event' in k:
                        continue
                    else:
                        try:
                            mc[i][j][k] = ak.firsts(mc[i][j][k])
                        except:
                            continue
        else:
            for j in mc[i].fields:
                if 'event' in j:
                    continue
                else:
                    try:
                        mc[i][j] = ak.firsts(mc[i][j])
                    except:
                        continue
    return mc

In [11]:
ecf_list = dak.from_parquet('/cms/cephfs/data/store/user/cmoore24/skims/old/full_skims/nolepton/mc/hgg/new*').groomed_ecfs.fields

In [12]:
ratios = []
for i in range(len(ecf_list)):
    if ecf_list[i][2] == 'e':
        n1 = int(ecf_list[i][3])
        a = int(ecf_list[i][:2])
    else:
        n1 = int(ecf_list[i][2])
        a = int(ecf_list[i][0])
    for j in range(len(ecf_list)):
        if ecf_list[i] == ecf_list[j]:
            continue
        if ecf_list[j][2] == 'e':
            n2 = int(ecf_list[j][3])
            b = int(ecf_list[j][:2])
        else:
            n2 = int(ecf_list[j][2])
            b = int(ecf_list[j][0])
        if n1 < n2:
            continue
        else:
            beta1 = float(ecf_list[i][-2:])
            beta2 = float(ecf_list[j][-2:])
            exponent = (a * beta1) / (b * beta2)
            ratios.append(f'{ecf_list[i]}/{ecf_list[j]}**{exponent}')

In [13]:
upper = 1000
lower = 450
IL = 44.99

In [14]:
def ecf_hist(dataset, ecf_min, ecf_max):
    make_hist = hist.Hist.new.Reg(40, ecf_min, ecf_max, name='ECF', label='MC ECF').Weight()
    make_hist.fill(ECF=dataset)
    return make_hist

In [15]:
def build_weighted_ks_input(mc, data, xsecs, totals, lumi):
    sim_values = []
    sim_weights = []
    
    for sample, arr in mc.items():
        if len(arr) == 0 or sample not in xsecs or sample not in totals:
            continue
        
        xsec = xsecs[sample]
        nevt = totals[sample]
        weight = (xsec * lumi) / nevt

        score_vals = ak.to_numpy(arr)
        sim_values.append(score_vals)
        sim_weights.append(np.full(len(score_vals), weight))


    sim_values = np.concatenate(sim_values)
    sim_weights = np.concatenate(sim_weights)

    data_values = []

    vals = ak.to_numpy(data)
    data_values.append(vals)
    data_values = np.concatenate(data_values)

    return data_values, sim_values, sim_weights

In [16]:
def make_weighted_ecdf(values, weights):
    sorter = np.argsort(values)
    sorted_vals = values[sorter]
    sorted_weights = weights[sorter]
    cumsum = np.cumsum(sorted_weights)
    cdf_vals = cumsum / cumsum[-1]

    def cdf_func(x):
        return np.interp(x, sorted_vals, cdf_vals, left=0.0, right=1.0)

    return cdf_func

In [24]:
ecf_type = 'groomed'

In [25]:
path = '/cms/cephfs/data/store/user/cmoore24/skims/full_skims/singlemuon/mc/2017'
columns=['goodjets.msoftdrop', 'goodjets.pt', f"{ecf_type}_ecfs"]
mc = read_files(f'{path}')
mc['ww'] = mc['diboson_ww']
mc['wz'] = mc['diboson_wz']
mc['zz'] = mc['diboson_zz']
del(mc['diboson_ww'])
del(mc['diboson_wz'])
del(mc['diboson_zz'])
del(mc['qcd_300to470'])
mc = firsts(mc)

path = '/cms/cephfs/data/store/user/cmoore24/skims/full_skims/singlemuon/data/2017'
data = read_files(f'{path}')
data = firsts(data)

for i in xsecs:
    try:
        if type(mc[i]) == dict:
            for j in mc[i]:
                mask = ((mc[i][j].goodjets.pt >= lower) & (mc[i][j].goodjets.pt <= upper))
                mc[i][j] = mc[i][j][mask]
        else:
            mask = ((mc[i].goodjets.pt >= lower) & (mc[i].goodjets.pt <= upper))
            mc[i] = mc[i][mask]
    except:
        continue

for i in data:
    if type(data[i]) == dict:
        for j in data[i]:
            mask = ((data[i][j].goodjets.pt >= lower) & (data[i][j].goodjets.pt <= upper))
            data[i][j] = data[i][j][mask]
    else:
        mask = ((data[i].goodjets.pt >= lower) & (data[i].goodjets.pt <= upper))
        data[i] = data[i][mask]

data_arr = ak.concatenate([data[i] for i in data])

In [19]:
ecfs_ks = {}
with open(f'{ecf_type}_ratio_ks.json','w') as f:
    json.dump(ecfs_ks, f)

In [20]:
with open(f'{ecf_type}_ratio_ks.json','r') as f:
    ecf_ks = json.load(f)
print(len(ecf_ks))

for k in range(len(ecf_ks), len(ratios)):

    numerator, denominator = get_num_den(ratios[k])

    mc_ratios = {}
    for i in mc:
        if type(mc[i]) == dict:
            for j in mc[i]:
                mc_ratios[j] = add_ratio(ratios[k], mc[i][j][f'{ecf_type}_ecfs'])
        else:
            mc_ratios[i] = add_ratio(ratios[k], mc[i][f'{ecf_type}_ecfs'])

    data_ratio = add_ratio(ratios[k], data_arr[f'{ecf_type}_ecfs'])

    data_vals, sim_vals, sim_wts = build_weighted_ks_input(mc_ratios, data_ratio, xsecs, totals, IL)
    cdf_sim = make_weighted_ecdf(sim_vals, sim_wts)
    ks_result = kstest(data_vals, cdf=cdf_sim)

    with open(f'{ecf_type}_ratio_ks.json','r') as f:
        ecf_ks = json.load(f)

    result = {'statistic':ks_result.statistic, 'p-value':ks_result.pvalue, 'adjusted':-math.log10(ks_result.statistic)}
    
    ecf_ks[ratios[k]] = result
        
    with open(f'{ecf_type}_ratio_ks.json','w') as f:
        json.dump(ecf_ks, f)
        
    gc.collect()
    if k % 100 == 0:
        print(k)

0
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300


In [26]:
mc['wlnu_800to1200'].groomed_ecfs['1e205']

<Array [0.195, 0.172, 0.234, ..., 0.0883, 0.183, 0.206] type='270 * ?float64'>

In [23]:
mc['wlnu_800to1200'].ungroomed_ecfs['1e205']

<Array [0.195, 0.172, 0.234, ..., 0.0883, 0.183, 0.206] type='270 * ?float64'>